##### EEEL 4220 Final Project
## Wind Farm Sizing and Siting on IEEE-14 System

In [54]:
# Import libraries
import cvxpy as cp
import numpy as np
import pandas as pd

In [55]:
# Load data
b_df = pd.read_csv('buses.csv')
g_df = pd.read_csv('generators.csv')
l_df = pd.read_csv('branches.csv')
d_df = pd.read_csv('demand_scenarios.csv')
cfs_df = pd.read_csv('wind_cf_scenarios.csv')
y_df = pd.read_csv('wind_candidates.csv')

In [56]:
# Parameters
c0 = g_df['c0'].to_numpy().reshape(len(g_df),1) # quadratic cost coefficients for generator g
c1 = g_df['c1'].to_numpy().reshape(len(g_df),1) # linear cost coefficients for generator g
c2 = g_df['c2'].to_numpy().reshape(len(g_df),1) # fixed cost coefficients for generator g
Pmin = g_df['Pmin_MW'].to_numpy() # minimum generation limits for generator g
Pmax = g_df['Pmax_MW'].to_numpy() # maximum generation limits for generator g
x = l_df['x_pu'].to_numpy() # reactance of transmission line l
fmax = l_df['rateA_MW'].to_numpy() # thermal limit of line l
D = d_df.pivot(index='bus_id',columns='scenario_id',values='Pd_MW').to_numpy() # demand at bus b in scenario s
CF = cfs_df['capacity_factor'].to_numpy() # wind capacity factor in scenario s
omega = cfs_df['probability'].to_numpy() # probability (weight) of scenario s
ybar = pd.merge(b_df, y_df, on='bus_id', how='left')['max_capacity_MW'].fillna(0).to_numpy() # maximum installable wind capacity at bus b (0 if not candidate)
Cinv = 3.5 # investment cost per MW of installed wind capacity ($/MW)

# Decision variables
p = cp.Variable((len(g_df),len(cfs_df)), nonneg=True) # power output of generator g under scenario s
f  = cp.Variable((len(l_df),len(cfs_df))) # power flow on line l under scenario s
theta = cp.Variable((len(b_df),len(cfs_df))) # voltage angle at bus b (radians)
w = cp.Variable((len(b_df),len(cfs_df)), nonneg=True) # wind generation at bus b under scenario s
y = cp.Variable((len(b_df),1), nonneg=True) # installed wind capacity at bus b (MW)

In [57]:
# Initialize incidence matrices
# Generator–bus
# Ggb = 1 if generator g is connected to bus b, 0 otherwise.
G = np.zeros((len(g_df), len(b_df)))

# Line–bus
# Alb = 1 if bus b is the sending (“from”) end of line l, if bus b is the receiving (“to”) end of line l, 0 otherwise.
A = np.zeros((len(l_df), len(b_df)))

# Populate incidence matrices
# Generator–bus
for gen_index in range(len(g_df)):
    bus_id = g_df.loc[gen_index, 'bus_id']
    bus_index = bus_id - 1
    G[gen_index, bus_index] = 1

# Line–bus
for line_index in range(len(l_df)):
    from_bus_id = l_df.loc[line_index, 'from_bus']
    to_bus_id = l_df.loc[line_index, 'to_bus']
    from_bus_index = from_bus_id - 1
    to_bus_index = to_bus_id - 1
    A[line_index, from_bus_index] = 1
    A[line_index, to_bus_index] = -1

In [58]:
# Define constraints
# Initialize an empty constraint set
con = [] 

# power balance
con.append((A.T @ f) + D == (G.T @ p) + w)

# generator output limits
for s in range(len(cfs_df)):
    con.append(p[:,s] <= Pmax)  # maximum generation

# set reference angle
con.append(theta[0,:] == 0)

# # DC power flow
for s in range(len(cfs_df)):    
    con.append(f[:,s] == (A @ theta[:,s]) / x)

# line flow limits
for s in range(len(cfs_df)):
    con.append(f[:,s] <= fmax)  # maximum line flow
    con.append(f[:,s] >= -fmax)  # minimum line flow

# wind generation limits
con.append(w <= y @ CF.reshape(1,len(cfs_df)))  # wind generation cannot exceed installed capacity

# wind capacity limits
con.append(y <= ybar)  # maximum installed capacity

In [59]:
# Define objective function - total cost
obj = cp.Minimize(cp.sum(cp.multiply(omega, cp.sum(cp.multiply(c2, cp.square(p)) + cp.multiply(c1, p) + c0, axis=0))) 
                  + Cinv * cp.sum(y))

In [60]:
# Solve the problem
prob1 = cp.Problem(obj, con)
prob1.solve(solver = "HIGHS")
prob1.solve();

/opt/anaconda3/envs/PyEnv/lib/python3.13/site-packages/cvxpy/reductions/solvers/solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


In [61]:
# results
print("\n The total generation in the system is:")
print((p.value.sum(axis=0) + w.value.sum(axis=0)))
print("The total demand in the system is:") 
print(D.sum(axis=0))

# print("\n")
# print(D)
# print(((G.T @ p.value) + w.value - (A.T @ f.value)).round(1))

print("\n Expected operating cost:")
print(obj.value)

print("\n Thermal generator dispatch results: ")
GT = pd.DataFrame(p.value.round(1), index = ['G1', 'G2', 'G3', 'G4', 'G5'])
print(GT.round(1))

print("\n Wind dispatch results: ")
WT = pd.DataFrame(w.value.round(1), index = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14'])
print(WT.round(1))


 The total generation in the system is:
[233.10000204 259.00000232 284.90000255 246.05000207 271.95000297]
The total demand in the system is:
[233.1  259.   284.9  246.05 271.95]

 Expected operating cost:
839.8811401266801

 Thermal generator dispatch results: 
       0     1     2     3     4
G1  79.1  88.4  97.8  83.8  93.1
G2  42.9  47.6  52.2  45.2  49.9
G3  43.5  49.1  54.7  46.3  51.9
G4  35.9  39.4  42.9  37.7  41.2
G5  31.7  34.5  37.3  33.1  35.9

 Wind dispatch results: 
       0    1    2    3    4
B1   0.0  0.0  0.0  0.0  0.0
B2   0.0  0.0  0.0  0.0  0.0
B3   0.0  0.0  0.0  0.0  0.0
B4   0.0  0.0  0.0  0.0  0.0
B5   0.0  0.0  0.0  0.0  0.0
B6   0.0  0.0  0.0  0.0  0.0
B7   0.0  0.0  0.0  0.0  0.0
B8   0.0  0.0  0.0  0.0  0.0
B9   0.0  0.0  0.0  0.0  0.0
B10  0.0  0.0  0.0  0.0  0.0
B11  0.0  0.0  0.0  0.0  0.0
B12  0.0  0.0  0.0  0.0  0.0
B13  0.0  0.0  0.0  0.0  0.0
B14  0.0  0.0  0.0  0.0  0.0
